In [1]:
!pip install textrank4zh

## TextRank使用方式

In [1]:
#-*- encoding:utf-8 -*-
from __future__ import print_function

import sys
try:
    reload(sys)
    sys.setdefaultencoding('utf-8')
except:
    pass

import codecs
from textrank4zh import TextRank4Keyword, TextRank4Sentence

text = codecs.open(u'C:\\Users\\BIG DATA\\Desktop\\test\\textRank_test.txt', 'r', 'utf-8').read()
tr4w = TextRank4Keyword()

tr4w.analyze(text=text, lower=True, window=2)   # py2中text必须是utf8编码的str或者unicode对象，py3中必须是utf8编码的bytes或者str对象

print( 'KeyWord：' )
for item in tr4w.get_keywords(20, word_min_len=1):
    print(item.word, item.weight)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.770 seconds.
Prefix dict has been built succesfully.


KeyWord：
cx 0.04737648949050154
兩者 0.018101398851539233
目前 0.017097010259842147
推測 0.016377448430785518
軸距 0.014594161897589712
來 0.014376359704159673
mm 0.014284866693891944
扶手 0.012676388897457275
單元 0.012676388897457275
柴油 0.012676388897457275
skyactiv 0.012676388897457275
透過 0.012464443782584624
市場 0.012380538662033017
規格 0.011976064804726311
於 0.01140976996337063
擁有 0.011373327014499278
發現 0.011018486959664278
報導 0.010394297932237971
終於 0.010394297932237971
車 0.010108918658589858


## 從MongoDB資料庫取出評論並取出摘要、關鍵字、關鍵句

In [47]:
from pymongo import MongoClient
IP = 'localhost'
client = MongoClient(IP, 27017)
# Select database
db = client.test
# Select collection
collection = db.test
datas = collection.find({"brand":"Toyota"})
for idx ,data in enumerate(datas):
    if idx >50 and idx <60:
        tr4w = TextRank4Keyword(stop_words_file ="E:\\PythonWin\\workspace\\dict\\uselessDict\\stopWords(0702).txt")
        tr4w.analyze(text=data['content'], lower=True, window=2) 
        print( 'Keyword：' )
        for item in tr4w.get_keywords(20, word_min_len=1):
            print(item.word, item.weight)
        print(data['content'])
        print()
        print( 'KeySentance：' )
        for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num= 1):
            print(phrase)

        tr4s = TextRank4Sentence()
        tr4s.analyze(text=data['content'], lower=False, source = 'all_filters')

        print()
        print( '摘要：' )
        for item in tr4s.get_key_sentences(num=3):
            print(item.index, item.weight, item.sentence)

Keyword：
買 0.07304127148447914
cc 0.06500800756960581
高速公路 0.06500800756960581
mpv 0.05237808358996872
說 0.051882387310747916
quest 0.051882387310747916
想 0.051751542271914655
馬自達 0.0496380424511774
未來 0.04889276425459187
toyota 0.04889276425459187
cx9 0.04743676124020198
previa 0.04743676124020198
mazda 0.04700337317460753
問 0.03431114098134409
開 0.03431114098134409
需要 0.03431114098134409
車 0.03431114098134409
預計 0.02894260776958881
聽 0.02873183490559545
nissan 0.02873183490559545
本來是想買馬自達MPV,預計5月才買,誰知現在早已賣光光...😩,又要重新考慮車子了....聽說未來會上市CX9,好像200萬左右吧,3500cc有問題想請問大家:1.我們家共6人,一般只開高速公路,需要買到3500cc的車嗎?2.如果以3500cc的車來比較,Nissan的Quest,Toyota的Previa,及Mazda的Cx9有何優缺點呢?

KeySentance：
聽說未來

摘要：
0 0.5873472660167567 本來是想買馬自達MPV,預計5月才買,誰知現在早已賣光光...😩,又要重新考慮車子了....聽說未來會上市CX9,好像200萬左右吧,3500cc有問題想請問大家:1.我們家共6人,一般只開高速公路,需要買到3500cc的車嗎
1 0.41265273398324315 2.如果以3500cc的車來比較,Nissan的Quest,Toyota的Previa,及Mazda的Cx9有何優缺點呢
Keyword：
油耗 0.05517695798332641
toyota 0.02539464340543785
變速 0.020729468399122368
camry 0.0169

In [40]:
for text in data['replies']:
    print(text)

In [9]:
print()
print( '关键短语：' )
for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num= 2):
    print(phrase)

tr4s = TextRank4Sentence()
tr4s.analyze(text=data['content'], lower=True, source = 'all_filters')

print()
print( '摘要：' )
for item in tr4s.get_key_sentences(num=3):
    print(item.index, item.weight, item.sentence)


关键短语：

摘要：
0 1.0 話說有天在街上亂晃時...突然看到前方讓我眼睛為之一亮😆LV耶😲性能~價格不知如何@@


Write the short review to MongoDB

In [ ]:
import sqlite3
from textrank4zh import TextRank4Sentence
from pymongo import MongoClient
#open mongoDB
IP ="localhost"
client = MongoClient(IP,27017)
db = client.test
collection = db.yahooShortReview

try:
    conn = sqlite3.connect("C:\\Users\\BIG DATA\\Desktop\\test\\yahoo_new_car.sqlite3")
    cursor = conn.cursor()
    # conn.close()

    for row in cursor.execute("select brand,model,years,type,url,intro from yahooNewCars;"):
        brand = row[0]
        model = row[1]
        years = row[2]
        type  = row[3]
        url   = row[4]
        intro = row[5]
        tr4s = TextRank4Sentence()
        tr4s.analyze(text=intro, lower=False, source = 'all_filters')
    #     print(brand,model,url,intro)
        for item in tr4s.get_key_sentences(num=1,sentence_min_len=10):
            review = {}
            review['brand'] = brand
            review['model'] = model
            review['years'] = years
            review['type'] = type
            review['url'] = url
            review['shortReview'] = item.sentence
            
            print(review)
            print("===============================================")
        db.yahooReview.insert_one(review)
            
    conn.commit()
except Exception as e:
    print("There is an exception on {}".format(url),e)
    
    pass
finally:
    conn.close()

{'brand': 'VOLKSWAGEN', 'model': 'PASSAT VARIANT', 'years': 2008, 'type': '2.0 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-passat-variant-2008-2-.-0-tdi', 'shortReview': '從外觀上來看，Passat Variant在B柱之前幾乎與Passat如出一轍，前座佈局也大致相同，但是後座空間則比起Passat增加了更多的變化，例如後座椅提供了6/4分離倒覆和傾倒躺平功能，因而也讓行李廂得以向前方延伸，讓旅行車的置物機能獲得最大發揮'}
{'brand': 'VOLKSWAGEN', 'model': 'T5', 'years': 2008, 'type': 'Shuttle SWB 2.5 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-t5-2008-shuttle-swb-2-.-5-tdi', 'shortReview': 'Volkswagen聞名於世的T系列箱型車，從第一代的T1發展到現今的T5，在傳承了近一甲子後Volkswagen已將Multivan獨立成第五世代，而今販售的車型在國內共包含Multivan、Caravelle、California、Shuttle等四種車型，而且也分別針對不同使用需求滿足包含商務、家庭、高階商旅等多種用途'}
{'brand': 'VOLKSWAGEN', 'model': 'T5', 'years': 2008, 'type': 'Shuttle LWB 1.9 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-t5-2008-shuttle-lwb-1-.-9-tdi', 'shortReview': 'Volkswagen聞名於世的T系列箱型車，從第一代的T1發展到現今的T5，在傳承了近一甲子後Volkswagen已將Multivan獨立成第五世代，而今販售的車型在國內共包含Multivan、Caravelle、California、Shuttle

{'brand': 'VOLKSWAGEN', 'model': 'PASSAT', 'years': 2008, 'type': '3.2 V6', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-passat-2008-3-.-2-v6', 'shortReview': 'Passat在Volkswagen旗下向來扮演著中大型豪華房車角色，除了外型採用典雅、成熟的設計風格外，在內裝部份也兼具了科技性與豪華、精緻要素，全車座椅更使用高檔的Alcantara/Napa，另外還有雙區全自動恆溫空調、Auto Hold車輛靜止功能、雨滴感應式雨刷、金屬質感與原木飾條組成的中控台、可供選配的12向電動駕駛座、駕駛座電動腰靠及HID主動式轉向氙氣大燈等，因而也為Passat在當今豪華房車領域中奠下了根基'}
{'brand': 'VOLKSWAGEN', 'model': 'PASSAT', 'years': 2008, 'type': '1.8 TSI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-passat-2008-1-.-8-tsi', 'shortReview': 'Passat在Volkswagen旗下向來扮演著中大型豪華房車角色，除了外型採用典雅、成熟的設計風格外，在內裝部份也兼具了科技性與豪華、精緻要素，全車座椅更使用高檔的Alcantara/Napa，另外還有雙區全自動恆溫空調、Auto Hold車輛靜止功能、雨滴感應式雨刷、金屬質感與原木飾條組成的中控台、可供選配的12向電動駕駛座、駕駛座電動腰靠及HID主動式轉向氙氣大燈等，因而也為Passat在當今豪華房車領域中奠下了根基'}
{'brand': 'VOLKSWAGEN', 'model': 'PASSAT', 'years': 2008, 'type': '2.0 TSI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-passat-2008-2-.-0-tsi', 'shortReview': 'Passat在Volkswagen旗下向來扮演著中大型豪華房車角色，除了外型採

{'brand': 'VOLKSWAGEN', 'model': 'T5', 'years': 2009, 'type': 'Multivan 2.5 TDI HL', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-t5-2009-multivan-2-.-5-tdi-hl', 'shortReview': 'Volkswagen聞名於世的T系列箱型車，從第一代的T1發展到現今的T5，在傳承了近一甲子後Volkswagen已將Multivan獨立成第五世代，而今販售的車型在國內共包含Multivan、Caravelle、California、Shuttle等四種車型，而且也分別針對不同使用需求滿足包含商務、家庭、高階商旅等多種用途'}
{'brand': 'VOLKSWAGEN', 'model': 'T5', 'years': 2009, 'type': 'Caravelle 2.5 TDI SWB', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-t5-2009-caravelle-2-.-5-tdi-swb', 'shortReview': 'Volkswagen聞名於世的T系列箱型車，從第一代的T1發展到現今的T5，在傳承了近一甲子後Volkswagen已將Multivan獨立成第五世代，而今販售的車型在國內共包含Multivan、Caravelle、California、Shuttle等四種車型，而且也分別針對不同使用需求滿足包含商務、家庭、高階商旅等多種用途'}
{'brand': 'VOLKSWAGEN', 'model': 'T5', 'years': 2009, 'type': 'Caravelle 2.5 TDI LWB', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-t5-2009-caravelle-2-.-5-tdi-lwb', 'shortReview': 'Volkswagen聞名於世的T系列箱型車，從第一代的T1發展到現今的T5，在傳承了近一甲子後Volkswagen已將Multivan獨立成第五世代，而今販售的車

{'brand': 'VOLKSWAGEN', 'model': 'TOURAN', 'years': 2010, 'type': '1.9 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-touran-2010-1-.-9-tdi', 'shortReview': 'Touran目前只引進1.9 TDI單一車型，搭載1.9升直四SOHC柴油引擎(最大輸出105hp/25.5kgm)，以及七速DSG雙離合器自手排變速系統，無論是全家滿載出遊、或是日常上下班通勤都是游刃有餘，上市以來也獲得廣大消費者的支持'}
{'brand': 'VOLKSWAGEN', 'model': 'TIGUAN', 'years': 2010, 'type': '2.0 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-tiguan-2010-2-.-0-tdi', 'shortReview': '國內目前引進的Tiguan共有兩種引擎可供選擇，一是2.0升TSI汽油引擎，擁有200hp/28.6kgm的強悍輸出實力（另有170匹低輸出版），另一是2.0升TDI柴油引擎，最大出力則達到140hp/32.6kgm，上述兩種引擎皆是搭配Tiptronic六速手自排變速系統，卓越的換檔效能與手排操作模式，絕對可滿足現代人的駕馭需求'}
{'brand': 'VOLKSWAGEN', 'model': 'CADDY MAXI', 'years': 2009, 'type': '1.9 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-caddy-maxi-2009-1-.-9-tdi', 'shortReview': '另外在安全配備上，雙前座SRS氣囊、防鎖死煞車系統（ABS）、電子防滑差速系統（EDL）以及加速/減速巡跡控制系統（ASR/MSR）等一應俱全，也讓Caddy Maxi得以充分展現MPV的全方位功能性，再加上其獨具都市機能的風格，絕對滿足消費者休閒與商務上的使用需求'}
{'brand': 'VOLKSWAGEN', 'model': '

{'brand': 'VOLKSWAGEN', 'model': 'GOLF', 'years': 2010, 'type': '1.6 TL', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-golf-2010-1-.-6-tl', 'shortReview': '目前導入的共有1.6、1.4 TSI兩款汽油引擎與1.6 TDI、2.0 TDI高壓柴油直噴引擎，高性能車款則有GTI以滿足消費者不同的需求，超極鋼砲代表的Golf R也在引進之列'}
{'brand': 'VOLKSWAGEN', 'model': 'GOLF', 'years': 2010, 'type': 'R 3D', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-golf-2010-r-3d', 'shortReview': '目前導入的共有1.6、1.4 TSI兩款汽油引擎與1.6 TDI、2.0 TDI高壓柴油直噴引擎，高性能車款則有GTI以滿足消費者不同的需求，超極鋼砲代表的Golf R也在引進之列'}
{'brand': 'VOLKSWAGEN', 'model': 'GOLF', 'years': 2010, 'type': '2.0 TDI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-golf-2010-2-.-0-tdi', 'shortReview': '目前導入的共有1.6、1.4 TSI兩款汽油引擎與1.6 TDI、2.0 TDI高壓柴油直噴引擎，高性能車款則有GTI以滿足消費者不同的需求，超極鋼砲代表的Golf R也在引進之列'}
{'brand': 'VOLKSWAGEN', 'model': 'GOLF', 'years': 2010, 'type': 'GTI', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/volkswagen-golf-2010-gti', 'shortReview': '目前導入的共有1.6、1.4 TSI兩款汽油引擎與1.6 TDI、2.0 TDI高壓柴油直噴引擎，

...

{'brand': 'TOYOTA', 'model': 'LAND CRUISER', 'years': 2015, 'type': 'Prado 4.0 VX', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-land-cruiser-2015-prado-4-.-0-vx', 'shortReview': 'Prado的車室內裝擁有高級車的質感與品味，而小改款後全車換裝深色木紋飾板，搭配座椅外側雙車縫線設計，更營造出典雅自信的氛圍'}
{'brand': 'TOYOTA', 'model': 'CAMRY NEW', 'years': 2015, 'type': 'Hybrid經典', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-camry-new-2015-hybrid經典', 'shortReview': '此外，CAMRY也提供三區獨立恆溫空調/後座椅電動調整(HYBRID旗艦)、觸控式影音/衛星導航主機附JBL Green Edge 10具鑑賞及揚聲器系統、nanoe奈米水離子空氣清淨系統、QI無線充電座等先進配備(HYBRID旗艦、HYBRID尊爵)，徹底滿足客戶對旗艦房車的配備需求'}
{'brand': 'TOYOTA', 'model': 'CAMRY NEW', 'years': 2015, 'type': '2.0豪華', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-camry-new-2015-2-.-0豪華', 'shortReview': '此外，CAMRY也提供三區獨立恆溫空調/後座椅電動調整(HYBRID旗艦)、觸控式影音/衛星導航主機附JBL Green Edge 10具鑑賞及揚聲器系統、nanoe奈米水離子空氣清淨系統、QI無線充電座等先進配備(HYBRID旗艦、HYBRID尊爵)，徹底滿足客戶對旗艦房車的配備需求'}
{'brand': 'TOYOTA', 'model': 'CAMRY NEW', 'years': 2015, 'type': '2.0經典', 'url': 'https://tw.autos.yahoo.c

{'brand': 'TOYOTA', 'model': 'VIOS', 'years': 2016, 'type': '1.5豪華', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-vios-2016-1-.-5豪華', 'shortReview': '以Toyota新世代「Keen Look」設計語彙為出發點的Vios，藉由廠徽向兩側延伸的鍍鉻水箱護罩與晶鑽頭燈線條結合，搭配盾型引擎蓋、倒梯形下氣壩設計，及由車側前後延伸的立體折線造型，勾勒出魅力十足的時尚動感新主張'}
{'brand': 'TOYOTA', 'model': 'VIOS', 'years': 2016, 'type': '1.5經典', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-vios-2016-1-.-5經典', 'shortReview': '以Toyota新世代「Keen Look」設計語彙為出發點的Vios，藉由廠徽向兩側延伸的鍍鉻水箱護罩與晶鑽頭燈線條結合，搭配盾型引擎蓋、倒梯形下氣壩設計，及由車側前後延伸的立體折線造型，勾勒出魅力十足的時尚動感新主張'}
{'brand': 'TOYOTA', 'model': 'RAV4', 'years': 2016, 'type': '2.5豪華', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-rav4-2016-2-.-5豪華', 'shortReview': '全新改款RAV4外觀設計更加科技動感，主被動安全配備全面升級，全車系搭載VSC車輛穩定控制、TRC循跡防滑控制及HAC上坡起步輔助系統，2.0豪華等級以上配備LED Bi-Beam頭燈及7SRS氣囊等，於同級距之中樹立全新標竿'}
{'brand': 'TOYOTA', 'model': 'RAV4', 'years': 2016, 'type': '2.5尊爵4WD', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-rav4-2016-2-.-5尊爵4wd', 'shortReview': '全新改款RAV4外觀

{'brand': 'TOYOTA', 'model': 'COROLLA ALTIS', 'years': 2016, 'type': '1.8經典版', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-corolla-altis-2016-1-.-8經典版', 'shortReview': 'Corolla Altis從超越顧客期待的理念出發，並以先進的歐規基準開發，提供更優越的機能與質感、以滿足駕馭的熱情與樂趣，Altis跳脫中型房車的框架，以超級距的大器感展現革命性的進化，車長、車寬及軸距都全面升級至中大型房車規格，車格更顯氣宇非凡，同時車室乘坐空間亦更加寬闊舒適'}
{'brand': 'TOYOTA', 'model': 'SIENTA', 'years': 2017, 'type': '7人座尊爵', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-sienta-2017-7人座尊爵', 'shortReview': '此外，安全性更是考量周全，全車搭載了九大主動安全系統*2，VSC(車輛穩定控制系統)、TRC(循跡防滑控制系統)、HAC(上坡起步輔助系統)、DSC(檔位誤入動力限制系統)、TPMS(胎壓偵測警示系統)、ABS(防鎖定煞車系統)、EBD(電子煞車力道分配系統)、BAS(煞車輔助系統)及BOS(煞車優先系統)，全面提升行車安全，讓車主安心感受Sienta所帶來的駕馭樂趣'}
{'brand': 'TOYOTA', 'model': 'COROLLA ALTIS', 'years': 2016, 'type': '1.8尊爵版', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-corolla-altis-2016-1-.-8尊爵版', 'shortReview': 'Corolla Altis從超越顧客期待的理念出發，並以先進的歐規基準開發，提供更優越的機能與質感、以滿足駕馭的熱情與樂趣，Altis跳脫中型房車的框架，以超級距的大器感展現革命性的進化，車長、車寬及軸距都全面升級至中大型房車規格，車格更顯氣宇非凡，同時車室乘坐空間亦更加寬闊舒適'}
{

{'brand': 'TOYOTA', 'model': 'CAMRY', 'years': 2017, 'type': 'Hybrid豪華', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-camry-2017-hybrid豪華', 'shortReview': '此外，CAMRY也提供三區獨立恆溫空調/後座椅電動調整(HYBRID旗艦)、觸控式影音/衛星導航主機附JBL Green Edge 10具鑑賞及揚聲器系統、nanoe奈米水離子空氣清淨系統、QI無線充電座等先進配備(HYBRID旗艦、HYBRID尊爵)，徹底滿足客戶對旗艦房車的配備需求'}
{'brand': 'TOYOTA', 'model': 'PREVIA', 'years': 2017, 'type': '2.4 Welcab', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-previa-2017-2-.-4-welcab', 'shortReview': '有感於近年消費者對於車身設計及安全配備的重視，2017 Previa特別推出「LX MODE尊榮外觀套件」打造全新外觀設計，同時全車系搭載PVM環景影像輔助系統，以及Drive+隨行駕駛系統附衛星導航，期望透過商品力強化，滿足尊榮車主的品味及行車需求'}
{'brand': 'TOYOTA', 'model': 'CAMRY', 'years': 2017, 'type': 'Hybrid尊爵', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/toyota-camry-2017-hybrid尊爵', 'shortReview': '此外，CAMRY也提供三區獨立恆溫空調/後座椅電動調整(HYBRID旗艦)、觸控式影音/衛星導航主機附JBL Green Edge 10具鑑賞及揚聲器系統、nanoe奈米水離子空氣清淨系統、QI無線充電座等先進配備(HYBRID旗艦、HYBRID尊爵)，徹底滿足客戶對旗艦房車的配備需求'}
{'brand': 'TOYOTA', 'model': 'CAMRY', 'years': 2017, 'type': '2.0豪華', '

{'brand': 'SUBARU', 'model': 'FORESTER', 'years': 2009, 'type': '2.5XT', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/subaru-forester-2009-2-.-5xt', 'shortReview': 'Forester目前在國內共引進2.0X以及2.5XT兩種車型，動力系統方面，2.5XT車型搭載的是2.5升水平對臥四缸渦輪增壓引擎，最大馬力可以達到230hp，最大扭力則有32.6kgm的水準'}
{'brand': 'SUBARU', 'model': 'FORESTER', 'years': 2009, 'type': '2.0X', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/subaru-forester-2009-2-.-0x', 'shortReview': 'Forester目前在國內共引進2.0X以及2.5XT兩種車型，動力系統方面，2.5XT車型搭載的是2.5升水平對臥四缸渦輪增壓引擎，最大馬力可以達到230hp，最大扭力則有32.6kgm的水準'}
{'brand': 'SUBARU', 'model': 'FORESTER', 'years': 2008, 'type': '2.5XT', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/subaru-forester-2008-2-.-5xt', 'shortReview': 'Forester目前在國內共引進2.0X以及2.5XT兩種車型，動力系統方面，2.5XT車型搭載的是2.5升水平對臥四缸渦輪增壓引擎，最大馬力可以達到230hp，最大扭力則有32.6kgm的水準'}
{'brand': 'SUBARU', 'model': 'FORESTER', 'years': 2008, 'type': '2.0X', 'url': 'https://tw.autos.yahoo.com/new-cars/trim/subaru-forester-2008-2-.-0x', 'shortReview': 'Forester目前在國內共引進2.0X以及2.5XT兩